# 1. importt libararies

In [19]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
from collections import OrderedDict

# 2. Data load

In [7]:
data_set = dset.MNIST('./data',train=True,transform=transforms.ToTensor(),
                     target_transform=None,download=True)

In [10]:
data = torch.utils.data.DataLoader(data_set, batch_size=128,shuffle=True,
                                          num_workers=0,drop_last=True)
print(len(data))

468


# 3. Model

In [105]:
# Generator
# [z+onehot vector] -> [28*28*1]
# z is vector of 100 rnadom numbers(0~1)
# one-hot vector size is 10

class G(nn.Module):
    def __init__(self,dim_z=100,dim_c = 10,input_size=28,out_dim=16,colch=1,leaky=0.2):
        super(G,self).__init__()
        
        self.dim_z = dim_z
        self.dim_c = dim_c
        self.input_size = input_size
        self.out_dim = out_dim
        self.colch = colch
        self.leaky = leaky
        
        
        self.fc = deconv(dim_c+dim_z,out_dim*16,int(input_size/4),1,0,bn=False) # [batch,110] -> [batch,256,7,7]
        self.layer1 = nn.Sequential(OrderedDict([ 
            ('conv1',nn.ConvTranspose2d(out_dim*16,out_dim*8,3,2,1)),  # [batch,256,7,7] -> [batch,128,14,14]
            ('relu1',nn.LeakyReLU(negative_slope=0.2)),
            ('bn1',nn.BatchNorm2d(128)),
            ('conv2',nn.ConvTranspose2d(out_dim*8,out_dim*4,3,1,1)) , # [batch,128,14,14] -> [batch,64,14,14]
            ('relu2',nn.LeakyReLU(negative_slope=0.2)),
            ('bn2',nn.BatchNorm2d(64))         
        ]))
        self.layer2 = nn.Sequential(OrderedDict([ 
            ('conv3',nn.ConvTranspose2d(out_dim*4,out_dim*1,3,2,1)),  # [batch,64,14,14] -> [batch,16,28,28]
            ('relu3',nn.LeakyReLU(negative_slope=0.2)),
            ('bn3',nn.BatchNorm2d(16)),
            ('conv4',nn.ConvTranspose2d(out_dim*1,1,3,1,1)) , # [batch,16,28,28] -> [batch,1,28,28]
            ('relu4',nn.LeakyReLU(negative_slope=0.2))   
        ]))

        
    def forward(self,z,c):
        z_c = torch.cat([z,c],1) 
        z_c = z_c.view(z_c.size(0),z_c.size(1),1,1)
        out = self.fc(z_c)
        out = self.layer1(out)
        out = self.layer2(out)
        out = F.tanh(out)
        return out

In [169]:
# Discriminator
# [28*28*1] -> output variable (real data or not), one-hot vector(label)
# output variable is one number(0~1)
# one-hot vector size is 10

class D(nn.Module):
    def __init__(self,input_size=28,colch=1,conv_dim=16):
        super(D,self).__init__()
        
        self.input_size = input_size
        self.colch = colch
        self.conv_dim = conv_dim
        
        self.layer1 = nn.Sequential(OrderedDict([
            ('conv1',nn.Conv2d(1,16,3,padding=1)), # [batch,1,28,28] -> [batch,16,28,28]
            ('relu1',nn.ReLU()),
            ('bn1',nn.BatchNorm2d(16)),
            ('conv2',nn.Conv2d(16,32,4,2,padding=0)), # [batch,16,28,28] -> [batch,32,14,14]
            ('relu2',nn.ReLU()),
            ('bn2',nn.BatchNorm2d(32))
        ]))
        self.layer2 = nn.Sequential(OrderedDict([
            ('conv3',nn.Conv2d(32,64,3,padding=1)), # [batch,32,14,14] -> [batch,64,14,14]
            ('relu3',nn.ReLU()),
            ('bn3',nn.BatchNorm2d(64)),
            ('conv4',nn.Conv2d(64,128,4,2,padding=0)), # [batch,64,14,14] -> [batch,128,7,7]
            ('relu4',nn.ReLU()),
            ('bn4',nn.BatchNorm2d(128)),
        ]))
        
        self.fc1 = nn.Sequential(nn.ConvTranspose2d(128,1,7,1,0),
                                nn.Sigmoid())
        self.fc2 = nn.Sequential(nn.Linear(128*7*7,10),
                                nn.ReLU())
        
    def forward(self,x):
        out = self.layer1(x)
        print(out.size())
        out = self.layer2(out)
        output = self.fc1(out).squeeze()
        print(out.size())
        flat = out.view(out.size(0),-1)
        print(flat)
        label = self.fc2(flat)
        return output, label

In [170]:
# put class instance on multi gpu

generator = G().cuda()
discriminator = D().cuda()

In [171]:
# put labels on multi gpu

ones_label = Variable(torch.ones(128,1)).cuda()
zeros_label = Variable(torch.zeros(128,1)).cuda()

In [172]:
# loss function and optimizer 
# this time, use LSGAN loss(https://arxiv.org/abs/1611.04076v2)

loss_func = nn.MSELoss()
gen_optim = torch.optim.Adam(generator.parameters(), lr=0.01)
dis_optim = torch.optim.Adam(discriminator.parameters(), lr=0.01)

In [173]:
# Define integer to onehot vector function
# 3 -> [0 0 0 1 0 0 0 0 0 0]

def int_to_onehot(z_label):
    one_hot_array = np.zeros(shape=[len(z_label), discrete_latent_size])
    one_hot_array[np.arange(len(z_label)), z_label] = 1
    return one_hot_array

In [174]:
# Set Hyperparameters

epoch = 1000
batch_size = 128
learning_rate = 0.001
num_gpus = 1
discrete_latent_size = 10

In [175]:
# train 

for i in range(100):
    for j,(image,label) in enumerate(data):
        
        # put image & label on gpu
        
        image = Variable(image).cuda()
        label = torch.from_numpy(int_to_onehot(label.numpy()))
        label = Variable(label.type_as(torch.FloatTensor())).cuda()
    
        # generator 
        
        for k in range(2):
            z_random = np.random.rand(batch_size,100)
            z_label = np.random.randint(0, 10, size=batch_size)
            z_label_onehot = int_to_onehot(z_label)
            
            # change first 10 labels from random to 0~9          
            for l in range(10):
                z_label[l]=l
            
            # preprocess z
            z_random = torch.from_numpy(z_random).type_as(torch.FloatTensor())
            z_random = Variable(z_random).cuda()
            z_label_onehot = torch.from_numpy(z_label_onehot).type_as(torch.FloatTensor())
            z_label_onehot = Variable(z_label_onehot).cuda()

            # calculate loss and apply gradients
            # gen_loss = gan loss(fake) + categorical loss
            gen_optim.zero_grad()
            gen_fake = generator.forward(z_random,z_label_onehot)
            dis_fake,label_fake = discriminator.forward(gen_fake)
            gen_loss = torch.sum(loss_func(dis_fake,ones_label)) \
                      + discrete_latent_size * torch.sum(loss_func(label_fake,z_label_onehot))
            gen_loss.backward(retain_variables=True)
            gen_optim.step()

        # discriminator
        # dis_loss = gan_loss(fake & real) + categorical loss
        dis_optim.zero_grad()
        dis_real, label_real = discriminator.forward(image)
        dis_loss = torch.sum(loss_func(dis_fake,zeros_label)) \
                  + torch.sum(loss_func(dis_real,ones_label)) \
                  + discrete_latent_size * torch.sum(loss_func(label_real,label))
        dis_loss.backward()
        dis_optim.step()
    
    # model save
        

torch.Size([128, 32, 11, 11])
torch.Size([128, 128, 4, 4])
Variable containing:
 7.5565e-02  1.2153e-02  7.1812e-02  ...  -2.1617e-01  2.0515e-01 -2.1617e-01
-8.8837e-02 -8.8837e-02  1.4595e-01  ...  -1.1095e-01  7.2284e-02 -2.1617e-01
-8.8837e-02 -8.8837e-02 -8.8837e-02  ...   9.6627e-03 -2.1617e-01 -2.1617e-01
                ...                   ⋱                   ...                
-3.3884e-02 -5.4901e-02 -4.7375e-02  ...   9.1787e-01  1.0589e+00 -2.1617e-01
-8.8837e-02 -8.8837e-02 -1.4540e-02  ...  -2.1617e-01 -2.1617e-01 -2.1617e-01
-6.7852e-02 -8.8837e-02 -8.8837e-02  ...   5.1000e-02  2.8283e-01 -2.1617e-01
[torch.cuda.FloatTensor of size 128x2048 (GPU 0)]



RuntimeError: size mismatch at d:\projects\pytorch\torch\lib\thc\generic/THCTensorMathBlas.cu:243

In [ ]:
z_label_onehot.size()

In [117]:
batch_size=128

In [161]:
128*49

6272